In [12]:
! pip install pycuda

In [13]:
import numpy as np
import time

import pycuda.gpuarray as gpuarray
import pycuda.autoinit
from  pycuda import  driver
from pycuda.compiler import SourceModule

# Алгоритм GPU

- Есть две последовательности: $x, y$ ;
- Считаем $temp = x^2 + y^2 $ ;
- Если $temp < 1 $, то возвращаем 1 (к счётчику прибавляем 1), иначе возвращаем 0 (оставляем значение счётчика без изменений);
- Выполняется атомарная операция сложения;
- Домножение на $4/n$ будет осуществлено позже.


In [14]:
mod = SourceModule("""
  __global__ void pi_mc_calc_gpu(double *x, double *y, double *result_gpu, const int n) {
        
        int gpu_count = 0;
        int idx = threadIdx.x + (blockIdx.x*blockDim.x);
        int thread_count = gridDim.x*blockDim.x;

        for (int i=idx; i<n; i += thread_count) {
          int temp;
          temp = pow(x[i], 2) + pow(y[i], 2);
          if (temp < 1)
            gpu_count += 1;
          
        }

        atomicAdd(result_gpu, gpu_count);
  }    
""")

# Алгоритм CPU

- На вход подаются также две последовательности $x,y$ (и число точек $n$);
- Считаем $temp = x^2 + y^2 $ ;
- Если $temp < 1 $, то возвращаем 1 (к счётчику прибавляем 1), иначе возвращаем 0 (оставляем значение счётчика без изменений);
- Умножение на $4/n$.

In [15]:
cpu_count = 0

def pi_mc_calc_cpu(x, y, n):
  temp = x ** 2 + y ** 2
  gen_cpu = [cpu_count + 1 for i in range(n) if temp[i] < 1]
  result = 4/n * sum(gen_cpu)
  return result

Так как в задании указано, что "Gain basic experience in writing CUDA programs and using CURAND library", то массивы x и y генерируются на GPU для GPU-вычислений, для CPU-вычислений массивы преобразовываются в ndarray функцией get().

In [16]:
 from pycuda.curandom import rand as curand

def generate_data():
  print('Введите число точек: ')
  n = int(input())
  assert n % 16 ==  0, 'вводимое число должно быть кратно 16'
  print('Введенное число точек: ', n)

  x_gpu = curand((n,), dtype=np.double) 
  y_gpu = curand((n,), dtype=np.double)
  x = x_gpu.get().astype(np.double)
  y = y_gpu.get().astype(np.double)
  print('Сгенерированные массивы: \n', x, '\n', y)
  return x, y, n

In [17]:
x, y, n = generate_data()

Введите число точек: 
16777216
Введенное число точек:  16777216
Сгенерированные массивы: 
 [0.35224954 0.02405453 0.54876547 ... 0.1193281  0.23744568 0.35254925] 
 [0.58437081 0.88769675 0.8967798  ... 0.5779781  0.61378046 0.65977915]


In [18]:
cpu_start = time.time()
result_cpu = pi_mc_calc_cpu(x,y, n)
cpu_time = time.time() - cpu_start
print('Число pi: ', result_cpu)
print('Время на CPU: ', round(cpu_time,4))

Число pi:  3.1418874263763428
Время на CPU:  6.2478


In [19]:
# т.к. массивы точек являются одномерными
block = (128, 1, 1)
grid = (int(n/(128 * block[0])), 1)

result_gpu = gpuarray.zeros((1,), dtype=np.double)
result_gpu  = result_gpu.get()

In [20]:
calc_gpu = mod.get_function("pi_mc_calc_gpu")

gpu_start = time.time()
calc_gpu(driver.In(x), driver.In(y), driver.Out(result_gpu), np.int32(n), block = block, grid = grid)
driver.Context.synchronize()
gpu_time = time.time() - gpu_start

result_gpu =  result_gpu[0] * 4/n
print('Число pi: ', result_gpu)
print('Время на GPU: ', round(gpu_time,4))

Число pi:  3.1418874263763428
Время на GPU:  0.135


In [21]:
print('Ускорение: ', cpu_time/gpu_time)
print('Сравнение с числом pi: ')
print('GPU:', abs(np.pi -  result_gpu) )
print('CPU:', abs(np.pi -  result_cpu) )

Ускорение:  46.26636075854386
Сравнение с числом pi: 
GPU: 0.00029477278654965744
CPU: 0.00029477278654965744
